# refinde code from here

## detector api

In [1]:
import torch, torchvision

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random, sys

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
sys.path.insert(1, '/home/appuser/beyless_train_header/dataset')
import register_custom_dataset as customDataset


class DTAPI:
    def __init__(self, model_path, weight_path):
        # model configuration
        customDataset.regist_custom_dataset('/home/appuser/beyless_train_header/dataset/')
        self.cfg = get_cfg()
        self.cfg.merge_from_file(model_path)
        self.cfg.MODEL.WEIGHTS = weight_path
        
        # post hyper parameter configuration
        self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
        self.cfg.MODEL.ROI_HEADS.NUM_CLASSES = 15

        # create default predictor
        self.predictor = DefaultPredictor(self.cfg)
    
    def inference(self, image_in_byte):
        self.raw_result = self.predictor(im)['instances']     
        return self.raw_result
    
    
    

In [ ]:
path = '/home/appuser/beyless_train_header/results/faster_rcnn_R_101_C4_3x.yaml_1604398136.6431131/'

## gRPC server

In [4]:
from concurrent import futures
import logging
import time
import grpc

import NormalDetect_pb2
import NormalDetect_pb2_grpc
from detectron2.external_functions.category_disabler import category_disabler

class beyless_detector(NormalDetect_pb2_grpc.NormalDetectorServicer):
    def __init__(self):
        self.call_counter = 1
        self.detector = DTAPI(_model_path, _weight_path)
        self.label_in_str = MetadataCatalog.get("custom_train").thing_classes
        category_disabler.cat_all_regist(self.label_in_str)

    def detect(self, ImageRequest, context):
        category_disabler.cat_in_regist(ImageRequest.productCodes)
        t0 = time.time()
        detection = self.detector.inference(ImageRequest.data)
        t1 = time.time()
        
        # create return form
        result = NormalDetect_pb2.DetectResponse()
        result.requestId = ImageRequest.requestId
        result.usedTime = t1 - t0
        result.status = 1
        result.msg = 'hello'
        
        boxes = []
        for i in range(len(detection)):
            _box = detection[i].pred_boxes.tensor.cpu().numpy().astype(np.int32).tolist()[0]
            _label = self.label_in_str[detection[i].pred_classes.cpu().numpy().astype(np.int32).tolist()[0]]
            _score = detection[i].scores.cpu().numpy().tolist()[0]
            data = NormalDetect_pb2.NormalBox(xmin = _box[0],
                                              ymin = _box[1], 
                                              xmax = _box[2], 
                                              ymax = _box[3], 
                                              label = _label, 
                                              score = _score )
            
            result.normalBoxes.append(data)
            
        return result

    
def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    NormalDetect_pb2_grpc.add_NormalDetectorServicer_to_server(beyless_detector(), server)
    server.add_insecure_port('[::]:50051')
    server.start()
    server.wait_for_termination()
    
    
if __name__ == '__main__':
    serve()


<<<<<< category disabler activated >>>>>>


/home/appuser/detectron2_repo/detectron2/modeling/roi_heads/fast_rcnn.py:129: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  filter_inds = filter_mask.nonzero()


<<<<<< category disabler activated >>>>>>
<<<<<< category disabler activated >>>>>>


KeyboardInterrupt: 